## ODM timewise

In [8]:
import pandas as pd
import os

ETM_folder = "Latest_ETM_Data"
ODM_timewise = "ODM_Timewise"

In [9]:
start_time = input("Enter the start time (HH:MM:SS format): ")
end_time = input("Enter the end time (HH:MM:SS format): ")

start_time = pd.to_datetime(start_time, format='%H:%M:%S')
end_time = pd.to_datetime(end_time, format='%H:%M:%S')

all_zonal_centroids = range(1, 3040)
all_pairs = pd.MultiIndex.from_product(
    [all_zonal_centroids, all_zonal_centroids],
    names=['source_zonal_centroid_number', 'destination_zonal_centroid_number']
)

for date in range(1, 2):
    date_str = f"{date:02}"
    try:
        df = pd.read_csv(f'./{ETM_folder}/Final_Ticket_data_2024-07-{date_str}.csv')
        
        #filter based on time
        df['time'] = pd.to_datetime(df['time'], format='%H:%M:%S')
        df_filtered = df[(df['time'] >= start_time) & (df['time'] <= end_time)]
        
        grouped = df_filtered.groupby(['source_zonal_centroid_number', 'destination_zonal_centroid_number'])['passenger_count'].sum()
        od_matrix = grouped.reindex(all_pairs, fill_value=0).unstack(fill_value=0)
        
        # Add Boarding and Alighting rows
        od_matrix['Boarding'] = od_matrix.sum(axis=1)
        od_matrix.loc['Alighting'] = od_matrix.sum(axis=0)

        # Save the OD matrix for the current day
        od_matrix.to_csv(f'{ODM_timewise}/ODM-07-{date_str}.csv', index=False)
        
        print(f"Processing file for 2024-07-{date_str} with time interval: {start_time.strftime('%H:%M:%S')} to {end_time.strftime('%H:%M:%S')}")

    except FileNotFoundError:
        print(f"File not found for 2024-07-{date_str}, skipping.")
    except Exception as e:
        print(f"An error occurred for 2024-07-{date_str}: {e}")

Enter the start time (HH:MM:SS format): 04:00:00
Enter the end time (HH:MM:SS format): 08:00:00
Enter the start date: 12
Enter the end date: 14
Processing file for 2024-07-12 with time interval: 04:00:00 to 08:00:00
Processing file for 2024-07-13 with time interval: 04:00:00 to 08:00:00
Processing file for 2024-07-14 with time interval: 04:00:00 to 08:00:00


In [10]:
od_matrix

destination_zonal_centroid_number,1,2,3,4,5,6,7,8,9,10,...,3031,3032,3033,3034,3035,3036,3037,3038,3039,Boarding
source_zonal_centroid_number,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2478
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3036,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3037,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3038,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
